<a href="https://colab.research.google.com/github/lorihe/Springboard-Capstone3---Transfer_Learning/blob/main/Game_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras import callbacks, optimizers
from tensorflow.keras.layers import Dense, Conv2D, GlobalAvgPool2D, Input
import numpy as np
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Game_Classification

/content/drive/MyDrive/Game_Classification


In [ ]:
def img_Data(dir_path, target_size, batch, class_lst, preprocessing):
  """
    Create a data generator using ImageDataGenerator for loading and augmenting images from a directory.

    Args:
        dir_path (str): Path to the directory containing the image files organized in subdirectories.
        target_size (tuple): A tuple specifying the target height and width of the images.
        batch (int): The batch size for training or inference.
        class_lst (list): List of class names. Subdirectories in 'dir_path' should be named after these classes.
        preprocessing (function, optional): A function that is applied to each input image for preprocessing.
            This function should take an input image (either a PIL image or a Numpy array) and return a
            preprocessed Numpy array. Default is None.

    Returns:
        DirectoryIterator: A DirectoryIterator yielding batches of augmented/loaded image data along with labels.
  """
  if preprocessing:
    gen_object = ImageDataGenerator(preprocessing_function = preprocessing)
  else:
    gen_object = ImageDataGenerator()

  return(gen_object.flow_from_directory(dir_path, target_size, batch_size = batch, class_mode = 'sparse',
                                   classes = class_lst, shuffle = True))

In [ ]:
train_data_gen = img_Data('clean_data/train', (224,224), 500, ['rugby','soccer'], preprocess_input)

Found 2448 images belonging to 2 classes.


In [ ]:
valid_data_gen = img_Data('clean_data/test', (224,224), 500, ['rugby','soccer'], preprocess_input)

Found 610 images belonging to 2 classes.


In [ ]:
#Use pre-trained weights to build the base model
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3), alpha=1.0, include_top=False, weights="imagenet",
    input_tensor=None, pooling=None, classes=2,
    classifier_activation="softmax"
)

9406464/9406464 [==============================] - 1s 0us/step


In [ ]:
base_model.trainable = False

In [ ]:
#Add customized layers
from keras.layers.attention.multi_head_attention import activation
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(GlobalAvgPool2D())
model.add(Dense(1024, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_1   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 1024)              1311744   
                                                                 
 dense_3 (Dense)             (None, 10)                10250     
                                                                 
Total params: 3,579,978
Trainable params: 1,321,994
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
elst = callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, mode = 'min')
save_ck = callbacks.ModelCheckpoint('model.hdf5', save_best_only = True, monitor = 'val_loss', mode = 'min')

In [ ]:
model.fit(train_data_gen, batch_size = 500, validation_data = valid_data_gen, callbacks = [elst, save_ck], epochs = 10)

Epoch 1/10
5/5 [==============================] - 58s 12s/step - loss: 0.4019 - accuracy: 0.8456 - val_loss: 0.4205 - val_accuracy: 0.8639
Epoch 2/10
5/5 [==============================] - 56s 12s/step - loss: 0.3626 - accuracy: 0.8742 - val_loss: 0.2944 - val_accuracy: 0.8770
Epoch 3/10
5/5 [==============================] - 55s 11s/step - loss: 0.2955 - accuracy: 0.8840 - val_loss: 0.3649 - val_accuracy: 0.8770
Epoch 4/10
5/5 [==============================] - 57s 12s/step - loss: 0.2477 - accuracy: 0.8995 - val_loss: 0.2678 - val_accuracy: 0.8852
Epoch 5/10
5/5 [==============================] - 64s 14s/step - loss: 0.2200 - accuracy: 0.9118 - val_loss: 0.3226 - val_accuracy: 0.8820
Epoch 6/10
5/5 [==============================] - 54s 11s/step - loss: 0.1959 - accuracy: 0.9195 - val_loss: 0.2527 - val_accuracy: 0.8984
Epoch 7/10
5/5 [==============================] - 56s 11s/step - loss: 0.1806 - accuracy: 0.9306 - val_loss: 0.2915 - val_accuracy: 0.9000
Epoch 8/10
5/5 [===========